In [8]:
with open('goethe/full.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [2]:
print(len(text))

5854235


In [4]:
print(text[:1000])

Wie froh bin ich, daß ich weg bin!  Bester Freund, was ist das Herz
des Menschen!  Dich zu verlassen, den ich so liebe, von dem ich
unzertrennlich war, und froh zu sein!  Ich weiß, du verzeihst mir's.
Waren nicht meine übrigen Verbindungen recht ausgesucht vom Schicksal,
um ein Herz wie das meine zu ängstigen?  Die arme Leonore!  Und doch
war ich unschuldig.  Konnt' ich dafür, daß, während die eigensinnigen
Reize ihrer Schwester mir eine angenehme Unterhaltung verschafften,
daß eine Leidenschaft in dem armen Herzen sich bildete?  Und doch--bin
ich ganz unschuldig?  Hab' ich nicht ihre Empfindungen genährt?  Hab'
ich mich nicht an den ganz wahren Ausdrücken der Natur, die uns so oft
zu lachen machten, so wenig lächerlich sie waren, selbst ergetzt?
Hab' ich nicht--o was ist der Mensch, daß er über sich klagen darf!
Ich will, lieber Freund, ich verspreche dir's, ich will mich bessern,
will nicht mehr ein bißchen Übel, das uns das Schicksal vorlegt,
wiederkäuen, wie ich's immer getan habe;

In [9]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"'()*+,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_abcdefghijklmnopqrstuvwxyz{}«»ÄÇÈÖÜßàâäèéêëòóöùûü—‘’“”„
114


In [10]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("Hallöchen"))
print(decode(encode("Hallöchen")))

[36, 59, 70, 70, 104, 61, 66, 63, 72]
Hallöchen


In [12]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([5854235]) torch.int64
tensor([51, 67, 63,  1, 64, 76, 73, 66,  1, 60, 67, 72,  1, 67, 61, 66,  9,  1,
        62, 59, 94,  1, 67, 61, 66,  1, 81, 63, 65,  1, 60, 67, 72,  2,  1,  1,
        30, 63, 77, 78, 63, 76,  1, 34, 76, 63, 79, 72, 62,  9,  1, 81, 59, 77,
         1, 67, 77, 78,  1, 62, 59, 77,  1, 36, 63, 76, 84,  0, 62, 63, 77,  1,
        41, 63, 72, 77, 61, 66, 63, 72,  2,  1,  1, 32, 67, 61, 66,  1, 84, 79,
         1, 80, 63, 76, 70, 59, 77, 77, 63, 72])


In [14]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [15]:
block_size = 8
train_data[:block_size+1]

tensor([51, 67, 63,  1, 64, 76, 73, 66,  1])

In [16]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
  # generate a small batch of data of inputs x and targets y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
  for t in range(block_size): # time dimension
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target: {target}")

In [23]:
from torch.nn import functional as F
import torch.nn as nn

In [30]:
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

#single head of self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2,-1) * head_size**-0.5 # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
out.shape


torch.Size([4, 8, 16])

In [31]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3966, 0.6034, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3069, 0.2892, 0.4039, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3233, 0.2175, 0.2443, 0.2149, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1479, 0.2034, 0.1663, 0.1455, 0.3369, 0.0000, 0.0000, 0.0000],
         [0.1259, 0.2490, 0.1324, 0.1062, 0.3141, 0.0724, 0.0000, 0.0000],
         [0.1598, 0.1990, 0.1140, 0.1125, 0.1418, 0.1669, 0.1061, 0.0000],
         [0.0845, 0.1197, 0.1078, 0.1537, 0.1086, 0.1146, 0.1558, 0.1553]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4016, 0.5984, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3365, 0.2271, 0.4364, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3019, 0.2060, 0.2899, 0.2022, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1058, 0.1700, 0.1530, 0.3451, 0.2261, 0.0000, 0.0000, 0.0000],
         [0.1526, 0.164